# Human Activity Recognition (HAR) Using Smartphones

Dataset contains Smartphone data of 30 subjects performing one of **6 activities**. <br>
**Multivariate timeseries data**, consisting of acceleromenter and gyroscope values. <br>
The task is **classifying** every timeseries to one of the acivities out of: **Walking**, **Walking-Upstairs**, **Walking-Doenstairs**, **Sitting**, **Standing**, **Laying**.

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# custom dataloader to handle the UCI HAR Dataset
from dataloader import UCIHARDataset, make_har_loaders

# Define Baseline CNN Model

In [3]:
class Baseline(nn.Module):
    """
    Simple CNN baseline:
      Conv1d -> ReLU -> MaxPool
      Conv1d -> ReLU -> MaxPool
      Global average pooling
      Linear -> Softmax  
    """
    def __init__(self, in_channels=6, num_classes=6):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv1d(in_channels, 32, kernel_size=7, padding=3),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),  # 128 -> 64

            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),  # 64 -> 32

            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
        )

        self.gap = nn.AdaptiveAvgPool1d(1)  # Global average pooling (batch, 128, 32) -> (batch, 128, 1)
        self.calssifier = nn.Linear(128, num_classes)

    def forward(self, x):
        # x: (batch, in_channels, time)
        x = self.features(x)
        x = self.gap(x).squeeze(-1)  # (batch, 128, 1) -> (batch, 128)
        logits = self.calssifier(x)       # (batch, num_classes)
        return logits